# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv("cities.csv")
weather_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,0,Norman Wells,75,CA,1633016108,100,65.2820,-126.8329,24.84,3.44
1,1,Bethel,90,US,1633016067,69,41.3712,-73.4140,62.89,8.05
2,2,Dorado,40,PR,1633016109,75,18.4588,-66.2677,90.18,5.75
3,3,Butaritari,100,KI,1633016110,73,3.0707,172.7902,81.66,11.61
4,4,Clyde River,90,CA,1633016110,75,70.4692,-68.5914,37.63,2.30
...,...,...,...,...,...,...,...,...,...,...
557,557,Kloulklubed,100,PW,1633016309,78,7.0419,134.2556,81.10,10.63
558,558,Tieli,7,CN,1633016310,85,46.9500,128.0500,39.27,5.48
559,559,Lubumbashi,40,CD,1633016310,33,-11.6609,27.4794,90.09,11.50
560,560,Araguaçu,30,BR,1633016310,26,-12.9306,-49.8264,100.69,5.88


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Long"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]

In [5]:
# Plot Heatmap
figure = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_humidity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
figure.add_layer(heat_layer)

# Display figure
print(figure)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
filtered_weather_df = weather_df.loc[(weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

filtered_weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
167,167,Huilong,0,CN,1633015983,81,31.8111,121.6550,73.92,2.62
274,274,Kashgar,0,CN,1633016225,14,39.4547,75.9797,76.21,6.71
334,334,Huoqiu,0,CN,1633016243,77,32.3394,116.2694,70.79,6.71
407,407,Pingdingshan,0,CN,1633016215,72,33.7385,113.3012,70.66,5.08
431,431,Aksu,0,CN,1633016270,16,41.1231,80.2644,71.85,9.35
440,440,Mananjary,0,MG,1633016273,80,-21.2167,48.3333,71.98,9.15
449,449,Kiryat Gat,0,IL,1633016108,100,31.6100,34.7642,79.39,8.19
486,486,Rosarito,0,MX,1633016287,65,32.3333,-117.0333,70.25,4.92


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
# Create a hotel_map_df
hotel_df= filtered_weather_df.filter(["City","Country", "Lat", "Long"], axis = 1)

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df 

,City,Country,Lat,Long,Hotel Name
167,Huilong,CN,31.8111,121.6550,
274,Kashgar,CN,39.4547,75.9797,
334,Huoqiu,CN,32.3394,116.2694,
407,Pingdingshan,CN,33.7385,113.3012,
431,Aksu,CN,41.1231,80.2644,
440,Mananjary,MG,-21.2167,48.3333,
449,Kiryat Gat,IL,31.6100,34.7642,
486,Rosarito,MX,32.3333,-117.0333,


In [28]:
# Api Call Link
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}





for index, row in hotel_df.iterrows():
    # Pull city name, lat, long from df
    lat = row["Lat"]
    long = row["Long"]
    city_name = row["City"]
    
    # Add keyword to parameters dict
    params["location"] = f"{lat},{long}"

    # Create Api request
    print(f"Preforming Api Call for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # Pull results
    results = response['results']
    
    # Save Hotel Name to df
    try:
        print(f"The Nearest Hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # Show entires with no hotels avalible  
    except (KeyError, IndexError):
        print("Sorry, No Hotel Availible...")
        
    print("**********")
    
    # Wait 20 sec to make another api request to avoid SSL Error
    time.sleep(20)

# End of Search Query Notification
print("***********Search Query Complete! ***********")

Preforming Api Call for Index 167: Huilong.
The Nearest Hotel in Huilong is Holiday Inn Express QIDONG DOWNTOWN.
**********
Preforming Api Call for Index 274: Kashgar.
The Nearest Hotel in Kashgar is Radisson Blu Hotel Kashgar.
**********
Preforming Api Call for Index 334: Huoqiu.
The Nearest Hotel in Huoqiu is Jiali Hotel.
**********
Preforming Api Call for Index 407: Pingdingshan.
The Nearest Hotel in Pingdingshan is Jiatian International Hotel.
**********
Preforming Api Call for Index 431: Aksu.
The Nearest Hotel in Aksu is Tianyuan International Hotel.
**********
Preforming Api Call for Index 440: Mananjary.
The Nearest Hotel in Mananjary is Jardin de la Mer.
**********
Preforming Api Call for Index 449: Kiryat Gat.
The Nearest Hotel in Kiryat Gat is Desert Gat.
**********
Preforming Api Call for Index 486: Rosarito.
The Nearest Hotel in Rosarito is Hotel Corona.
**********
***********Search Query Complete! ***********


In [30]:
hotel_df

,City,Country,Lat,Long,Hotel Name
167,Huilong,CN,31.8111,121.6550,Holiday Inn Express QIDONG DOWNTOWN
274,Kashgar,CN,39.4547,75.9797,Radisson Blu Hotel Kashgar
334,Huoqiu,CN,32.3394,116.2694,Jiali Hotel
407,Pingdingshan,CN,33.7385,113.3012,Jiatian International Hotel
431,Aksu,CN,41.1231,80.2644,Tianyuan International Hotel
440,Mananjary,MG,-21.2167,48.3333,Jardin de la Mer
449,Kiryat Gat,IL,31.6100,34.7642,Desert Gat
486,Rosarito,MX,32.3333,-117.0333,Hotel Corona


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [33]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
figure.add_layer(markers)

# Display Map
print(figure)

Figure(layout=FigureLayout(height='420px'))
